In [1]:
import pandas as pd
import requests
import urllib3
from datetime import datetime
from time import sleep

# SSL 경고 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

API_KEY = "524450d63e7f4a5b8292c7bd7484116b"  # 반드시 본인의 발급받은 API KEY를 입력하세요
BASE_URL = "https://www.schoolinfo.go.kr/openApi.do"

YEARS = [2022, 2023, 2024]
SCHOOL_TYPES = ["02", "03", "04"]  # 02: 초등, 03: 중등, 04: 고등
DEPTH_NO2_LIST = ["1", "2"]  # 1: 세입(예산), 2: 세출(예산)
DEPTH_NO = "10"  # 예산 (결산은 20)

# 학교회계 예결산서(국공립) API 호출 함수 (apiType=27)
def fetch_budget_data(pban_yr: int, schul_knd_code: str, depth_no2: str) -> pd.DataFrame:
    params = {
        "apiKey": API_KEY,
        "apiType": "27",
        "pbanYr": pban_yr,
        "schulKndCode": schul_knd_code,
        "depthNo": DEPTH_NO,
        "depthNo2": depth_no2
    }
    try:
        response = requests.get(BASE_URL, params=params, verify=False)
        response.raise_for_status()
        data = response.json()
        if data['resultCode'] != 'success':
            print(f"API Error: {data['resultMsg']} (year: {pban_yr}, type: {schul_knd_code}, depthNo2: {depth_no2})")
            return pd.DataFrame()
        df = pd.DataFrame(data['list'])
        df['연도'] = pban_yr
        df['학교급코드'] = schul_knd_code
        df['예산구분'] = '세입' if depth_no2 == '1' else '세출'
        return df
    except Exception as e:
        print(f"Error occurred: {str(e)} (year: {pban_yr}, type: {schul_knd_code}, depthNo2: {depth_no2})")
        return pd.DataFrame()

# 전체 데이터 수집
all_data = []

for year in YEARS:
    for school_type in SCHOOL_TYPES:
        for depth_no2 in DEPTH_NO2_LIST:
            print(f"Fetching year {year}, school type {school_type}, depthNo2 {depth_no2} ...")
            df = fetch_budget_data(pban_yr=year, schul_knd_code=school_type, depth_no2=depth_no2)
            if not df.empty:
                all_data.append(df)
            sleep(0.5)  # 서버 과부하 방지용 딜레이

# 결과 저장
if all_data:
    result_df = pd.concat(all_data, ignore_index=True)
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"학교회계_예산_세입세출_통합_{timestamp}.csv"
    result_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n✅ 전체 데이터 {filename}에 저장 완료! (총 {len(result_df)} rows)")
    print(result_df.head())
else:
    print("❌ 데이터가 수집되지 않았습니다.")


Fetching year 2022, school type 02, depthNo2 1 ...
Fetching year 2022, school type 02, depthNo2 2 ...
Fetching year 2022, school type 03, depthNo2 1 ...
Fetching year 2022, school type 03, depthNo2 2 ...
Fetching year 2022, school type 04, depthNo2 1 ...
Fetching year 2022, school type 04, depthNo2 2 ...
Fetching year 2023, school type 02, depthNo2 1 ...
Fetching year 2023, school type 02, depthNo2 2 ...
Fetching year 2023, school type 03, depthNo2 1 ...
Fetching year 2023, school type 03, depthNo2 2 ...
Fetching year 2023, school type 04, depthNo2 1 ...
Fetching year 2023, school type 04, depthNo2 2 ...
Fetching year 2024, school type 02, depthNo2 1 ...
Fetching year 2024, school type 02, depthNo2 2 ...
Fetching year 2024, school type 03, depthNo2 1 ...
Fetching year 2024, school type 03, depthNo2 2 ...
Fetching year 2024, school type 04, depthNo2 1 ...
Fetching year 2024, school type 04, depthNo2 2 ...

✅ 전체 데이터 학교회계_예산_세입세출_통합_20250518072422.csv에 저장 완료! (총 61856 rows)
  FOND_SC_CODE

In [7]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("학교회계_예산_세입세출_통합.csv")

# 조건: 서울특별시교육청 + 공립
filtered_df = df[
    (df['ATPT_OFCDC_ORG_NM'] == '서울특별시교육청') &
    (df['FOND_SC_CODE'] == '공립')
]

# 결과 확인
print(filtered_df.shape)
print(filtered_df.head())


(5754, 30)
  FOND_SC_CODE  SCHUL_KND_SC_CODE   ADRCD_NM       JU_ORG_NM  \
2           공립                  2  서울특별시 강남구  서울특별시강남서초교육지원청   
3           공립                  2  서울특별시 강남구  서울특별시강남서초교육지원청   
4           공립                  2  서울특별시 강남구  서울특별시강남서초교육지원청   
5           공립                  2  서울특별시 강남구  서울특별시강남서초교육지원청   
6           공립                  2  서울특별시 강남구  서울특별시강남서초교육지원청   

  ATPT_OFCDC_ORG_NM JU_ORG_CODE PBAN_EXCP_YN FAS_DTN_SC_CODE  \
2          서울특별시교육청  B100000249            N            복식부기   
3          서울특별시교육청  B100000249            N            복식부기   
4          서울특별시교육청  B100000249            N            복식부기   
5          서울특별시교육청  B100000249            N            복식부기   
6          서울특별시교육청  B100000249            N            복식부기   

  PBAN_REV_EX_SC_CODE      AMT6  ...  SCHUL_CRSE_SC_VALUE_NM  \
2                예산세입  20000000  ...                       초   
3                예산세입   8000000  ...                     NaN   
4                예산세입      

In [8]:
# 고정 파일명 설정
filename = "서울시_국공립_예산_세입세출.csv"

# CSV 저장
filtered_df.to_csv(filename, index=False, encoding='utf-8-sig')

print(f"✅ CSV 저장 완료: {filename}")


✅ CSV 저장 완료: 서울시_국공립_예산_세입세출.csv


In [3]:
filtered_df

,FOND_SC_CODE,SCHUL_KND_SC_CODE,ADRCD_NM,JU_ORG_NM,ATPT_OFCDC_ORG_NM,JU_ORG_CODE,PBAN_EXCP_YN,FAS_DTN_SC_CODE,PBAN_REV_EX_SC_CODE,AMT6,...,SCHUL_CRSE_SC_VALUE_NM,SCHUL_CRSE_SC_VALUE,PBAN_EXCP_RSN,연도,학교급코드,예산구분,AMT8,AMT7,YESAN_PER_HEAD,HS_KND_SC_NM
2,공립,2,서울특별시 강남구,서울특별시강남서초교육지원청,서울특별시교육청,B100000249,N,복식부기,예산세입,20000000,...,초,2.0,NaN,2022,2,세입,NaN,NaN,NaN,NaN
3,공립,2,서울특별시 강남구,서울특별시강남서초교육지원청,서울특별시교육청,B100000249,N,복식부기,예산세입,8000000,...,NaN,NaN,NaN,2022,2,세입,NaN,NaN,NaN,NaN
4,공립,2,서울특별시 강남구,서울특별시강남서초교육지원청,서울특별시교육청,B100000249,N,복식부기,예산세입,0,...,NaN,NaN,NaN,2022,2,세입,NaN,NaN,NaN,NaN
5,공립,2,서울특별시 강남구,서울특별시강남서초교육지원청,서울특별시교육청,B100000249,N,복식부기,예산세입,18000000,...,NaN,NaN,NaN,2022,2,세입,NaN,NaN,NaN,NaN
6,공립,2,서울특별시 강남구,서울특별시강남서초교육지원청,서울특별시교육청,B100000249,N,복식부기,예산세입,10000000,...,NaN,NaN,NaN,2022,2,세입,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60535,공립,4,서울특별시 강서구,서울특별시교육청,서울특별시교육청,B100000001,N,복식부기,예산세출,349152000,...,NaN,NaN,NaN,2024,4,세출,100000.0,20000000.0,1635571.9,일반고등학교
60536,공립,4,서울특별시 은평구,서울특별시교육청,서울특별시교육청,B100000001,N,복식부기,예산세출,300214000,...,고,4.0,NaN,2024,4,세출,0.0,0.0,1654020.2,일반고등학교
60537,공립,4,서울특별시 서대문구,서울특별시교육청,서울특별시교육청,B100000001,N,복식부기,예산세출,398091000,...,고,4.0,NaN,2024,4,세출,0.0,45000000.0,1678888.1,일반고등학교
60538,공립,4,서울특별시 성동구,서울특별시교육청,서울특별시교육청,B100000001,N,복식부기,예산세출,394007000,...,NaN,NaN,NaN,2024,4,세출,0.0,0.0,2113693.0,일반고등학교


In [4]:
import pandas as pd
import requests
import urllib3
from datetime import datetime
from time import sleep

# SSL 경고 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

API_KEY = "524450d63e7f4a5b8292c7bd7484116b"
BASE_URL = "https://www.schoolinfo.go.kr/openApi.do"

SCHOOL_TYPES = ["02", "03", "04"]  # 02: 초등, 03: 중등, 04: 고등

# 학교기본정보 API 호출 함수 (apiType=0)
def fetch_school_basic_info(schul_knd_code: str) -> pd.DataFrame:
    params = {
        "apiKey": API_KEY,
        "apiType": "0",
        "schulKndCode": schul_knd_code
    }
    try:
        response = requests.get(BASE_URL, params=params, verify=False)
        response.raise_for_status()
        data = response.json()
        if data['resultCode'] != 'success':
            print(f"API Error: {data['resultMsg']} (type: {schul_knd_code})")
            return pd.DataFrame()
        df = pd.DataFrame(data['list'])
        df['학교급코드'] = schul_knd_code
        return df
    except Exception as e:
        print(f"Error occurred: {str(e)} (type: {schul_knd_code})")
        return pd.DataFrame()

# 전체 데이터 수집
all_data = []

for school_type in SCHOOL_TYPES:
    print(f"Fetching school basic info for type {school_type} ...")
    df = fetch_school_basic_info(schul_knd_code=school_type)
    if not df.empty:
        all_data.append(df)
    sleep(0.5)  # 서버 과부하 방지용 딜레이

# 결과 저장
if all_data:
    result_df = pd.concat(all_data, ignore_index=True)
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"학교기본정보_초중고_통합_{timestamp}.csv"
    result_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n✅ 전체 데이터 {filename}에 저장 완료! (총 {len(result_df)} rows)")
    print(result_df.head())
else:
    print("❌ 데이터가 수집되지 않았습니다.")


Fetching school basic info for type 02 ...
Fetching school basic info for type 03 ...
Fetching school basic info for type 04 ...

✅ 전체 데이터 학교기본정보_초중고_통합_20250518084543.csv에 저장 완료! (총 12667 rows)
          SCHUL_RDNMA SCHUL_FOND_TYP_CODE FOAS_MEMRD DGHT_SC_CODE  \
0  서울특별시 서초구 서초중앙로 96                  부설   19530522           주간   
1    서울특별시 종로구 대학로 64                  부설   18950416           주간   
2    서울특별시 강남구 선릉로 29                  단설   19820920           주간   
3   서울특별시 강남구 개포로 401                  단설   19871117           주간   
4  서울특별시 강남구 삼성로4길 30                  단설   19830510           주간   

  FOND_SC_CODE USER_TELNO_SW      LTTUD ZIP_CODE SCHUL_KND_SC_CODE  \
0           국립  02-6009-6000  37.490739   137742                02   
1           국립   02-768-1514  37.577017   110810                02   
2           공립  02-2138-1940  37.481618   135810                02   
3           공립   02-571-8320  37.486214   135804                02   
4           공립  02-6952-4092  37.486805 

In [5]:
import pandas as pd

# 파일 불러오기
df_school = pd.read_csv("학교기본정보_초중고_통합.csv")

# 필터 조건: 서울특별시교육청 + 공립
filtered_school = df_school[
    (df_school['ATPT_OFCDC_ORG_NM'] == '서울특별시교육청') &
    (df_school['FOND_SC_CODE'] == '공립')
]

# 결과 확인
print(filtered_school.shape)
print(filtered_school[['SCHUL_NM', 'FOND_SC_CODE', 'ATPT_OFCDC_ORG_NM']].head())


(970, 39)
   SCHUL_NM FOND_SC_CODE ATPT_OFCDC_ORG_NM
2  서울개원초등학교           공립          서울특별시교육청
3  서울개일초등학교           공립          서울특별시교육청
4  서울개포초등학교           공립          서울특별시교육청
5  서울구룡초등학교           공립          서울특별시교육청
6  서울논현초등학교           공립          서울특별시교육청


In [6]:
# 고정 파일명 설정
filename = "서울시_국공립_학교기본정보.csv"

# CSV 저장
filtered_school.to_csv(filename, index=False, encoding='utf-8-sig')

print(f"✅ CSV 저장 완료: {filename}")


✅ CSV 저장 완료: 서울시_국공립_학교기본정보.csv
